In [1]:
from __future__ import print_function
import numpy as np
import numpy.linalg as LA
from scipy import optimize
from maxvolpy.maxvol import rect_maxvol, maxvol
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import gen_mat as gen
import grad_log_det as log_det # this is a module with analytical calculation of gradient and objective 
from test_bench import test_points_gen
from autograd import grad
from numba import jit, njit, prange
from test_bench import *
import os
%matplotlib inline

In [2]:
dim = 2
def OptPoints(num_col=50, num_points=54, num_tries=1, interval=(-1.0,1.0), poly=gen.cheb, useLHS=True, ToPlot=False):
    log_det.num_col = num_col # this is a global variable in module log_det
    log_det.dim = dim  
    bnds = (interval,) * (dim*num_points)
    dstrs = ['random']
    if useLHS:
        dstrs.append('LHS')
        
    loss_f = lambda x: log_det.loss_func(x, poly=poly, ToGenDiff=False)
    loss_grad = lambda x: log_det.grad(x, poly=poly)
    res = np.inf
    for distrib in dstrs:
        for _ in range(num_tries):
            x_0 = test_points_gen(num_points, dim, interval=interval, distrib=distrib) # starting point of GD
            x_0 = x_0.ravel('F')
            op = optimize.fmin_l_bfgs_b(loss_f, x_0, fprime = loss_grad, factr = 10.0, bounds = bnds)
            res_cur = op[1]
            if res_cur < res:
                res = res_cur
                pnts = op[0]
                opt_distr = distrib
            
    pnts = pnts.reshape(pnts.size//dim, dim, order="F")
    if ToPlot:
        ax = plt.figure(figsize=(7,7)).add_subplot(111)
        ax.plot(pnts[:,0], pnts[:,1], 'ro', label = "GD")
        ax.set_aspect(1)
                
    return res, pnts, opt_distr

def Experiment(dim=2, num_cols=(49, 50), num_points_max=54, interval=(-1.0, 1.0), num_tries=10, poly=gen.cheb, wdir='./res_GD', useLHS=True):
    try:
        os.mkdir(wdir)
    except:
        pass

    log_det.dim = dim # this is a global variable in module log_det
    for num_col in num_cols:
        log_det.num_col = num_col # this is a global variable in module log_det
        for num_points in  range(num_col, num_points_max):
            _, pnts, _ = OptPoints(num_col=num_col, 
                                   num_points=num_points, 
                                   num_tries=num_tries, 
                                   interval=interval, 
                                   poly=poly, 
                                   useLHS=useLHS,
                                   ToPlot=False)

            fn = os.path.join(wdir, "cols={}_points={}_poly={}".format(num_col, num_points, poly.__name__))
            np.savez(fn, pnts=pnts)   
        
    


In [ ]:
# Test OptPoints
min_det, pnts, distrib = OptPoints(num_col=36, 
                                   num_points=60, 
                                   num_tries=5, 
                                   interval=(-1.0, 1.0), 
                                   poly=gen.cheb, 
                                   useLHS=True,
                                   ToPlot=True)
print (min_det, distrib)

In [ ]:
NumOfClusters(pnts, full=False)

In [ ]:
np.random.seed(42)
test_pnts = test_points_gen(int(1e5), dim)
LebesgueConst(pnts, 36, poly=gen.legendre, test_pnts=test_pnts, pow_p=1)

In [ ]:
np.savez("pnts", pnts=pnts)

In [ ]:
pnts = np.load("pnts.npz")["pnts"]

In [ ]:
np.random.seed(42)
test_pnts = test_points_gen(int(1e5), dim)
ff = MakeValsAndNorms([f_gauss, f_sincos, f_rosenbrock], test_pnts)

In [ ]:
LebesgueConst(pnts, 36, poly=gen.legendre, test_pnts=test_pnts, pow_p=1, funcs=ff)

## Main experiment

In [ ]:
np.random.seed(2)
Experiment(dim=2, 
           num_cols=(36,), 
           num_points_max=37, 
           interval=(-1.0, 1.0), 
           num_tries=1, 
           poly=gen.cheb,
           useLHS=False,
           wdir='res_GD')
    

# Old Strategy

In [ ]:
# Workspace
"""
Here should be defined parameters of approximation:
– number of columns in model matrix (num_col)
– number of points using for approximation (num_points)
– dimension of the model (dim)
– bounds of domain (l_bound, u_bound)
– function to be approximated (def rhs(points))
"""
num_col = 50
num_points = 54

num_col = 36
num_points = 154

dim = 2

log_det.num_col = num_col # this is a global variable in module log_det
log_det.dim = dim         # this is a global variable in module log_det

assert num_col <= num_points, '! num_points < num_col !'

l_bound = -3.
u_bound = 3.

x_0 = test_points_gen(num_points, dim, interval=(l_bound, u_bound), distrib='random')  # starting point of GD
bnds = ((l_bound, u_bound),)* (dim*num_points)


    #return (1 - points[:,0])**2 + 100*(points[:,1] - points[:,0]**2)**2 # Rosenbrock
    #return (np.sin((points[:,0]**2)/2. - (points[:,1]**2)/4. + 3) * np.cos(2*points[:,0] + 1 - np.exp(points[:,1]))) # Trigonometric 

print('Max degree of expansion term = ', np.max(gen.indeces_K_cut(dim, num_col)))

In [ ]:
# optimization part (BFGS)
loss_f = lambda x: log_det.loss_func(x, ToGenDiff=False)
res = optimize.fmin_l_bfgs_b(loss_f, x_0, fprime = log_det.grad, factr = 10.0, bounds = bnds)
#res = optimize.fmin_l_bfgs_b(loss_f, x_0, fprime = grad(loss_f), factr = 10.0, bounds = bnds)

In [ ]:
# check of the correctness of analytical gradient calculation
def cg():
    print(optimize.check_grad(log_det.loss_func, log_det.grad, x_0, epsilon=epsilon))
    
epsilon = 1e-5
# print(optimize.check_grad(log_det.loss_func, grad(log_det.loss_func), x_0, epsilon=epsilon))
%time cg()

In [ ]:
pnts = res[0].reshape(res[0].size//dim, dim, order="F")

In [ ]:
# approximation error (infinity norm) and comparison with MaxVol
M_gd = gen.GenMat(num_col, pnts, poly = gen.cheb, ToGenDiff=False) 
c_gd = LA.solve(np.dot(M_gd.T, M_gd), np.dot(M_gd.T, rhs(pnts)))
_="""
np.random.seed(100500)
test = l_bound + (u_bound - l_bound)*np.random.rand(int(1e5), dim)

M = gen.GenMat(num_col, test, poly = gen.cheb, ToGenDiff=False) 

print("GD error = ", LA.norm(rhs(test) - np.dot(M, c_gd), np.inf) / LA.norm(rhs(test), np.inf), "\n")
# print ("rank =,", np.linalg.matrix_rank(M),", shape =", M.shape)
row_indx,_ = rect_maxvol(M, tol = 1.01, minK = num_points, maxK = num_points)

print ("MaxVol passed")

c_maxvol = LA.solve(np.dot(M[row_indx].T, M[row_indx]), np.dot(M[row_indx].T, rhs(test[row_indx])))

print("MaxVol error = ", LA.norm(rhs(test) - np.dot(M, c_maxvol), np.inf) / LA.norm(rhs(test), np.inf), "\n")


from sobol_lib import *
def GenSobol( N = 200, dim = 2, seed = 0, rng=(-1.0, 1.0) ):
    res = np.zeros((N, dim), dtype=float)
    rng_d = rng[1] - rng[0]
    for i in xrange(N):
        res[i, :], seed = i4_sobol ( dim, seed )
        res[i, :] = res[i, :]*rng_d + rng[0]

    return res

smp_sbl = GenSobol(num_points, dim, rng=(l_bound, u_bound))
M_sbl = gen.GenMat(num_col, smp_sbl, poly = gen.cheb, ToGenDiff=False)
c_sbl = LA.solve(np.dot(M_sbl.T, M_sbl), np.dot(M_sbl.T, rhs(M_sbl)))

print("Sobol error = ", LA.norm(rhs(test) - np.dot(M, c_sbl), np.inf) / LA.norm(rhs(test), np.inf), "\n")


# smp = l_bound + (u_bound - l_bound)*lhs(dim, num_points, criterion='m', iterations=100)
smp = l_bound + (u_bound - l_bound)*test_points_gen(num_points, dim, 'LHS')
M_lhs = gen.GenMat(num_col, smp, poly = gen.cheb, ToGenDiff=False)
c_lhs = LA.solve(np.dot(M_lhs.T, M_lhs), np.dot(M_lhs.T, rhs(smp)))

print("LHS error = ", LA.norm(rhs(test) - np.dot(M, c_lhs), np.inf) / LA.norm(rhs(test), np.inf), "\n")


smp_rndm = l_bound + (u_bound - l_bound)*np.random.rand(num_points, dim)
M_rndm = gen.GenMat(num_col, smp_rndm, poly = gen.cheb, ToGenDiff=False)
c_rndm = LA.solve(np.dot(M_rndm.T, M_rndm), np.dot(M_rndm.T, rhs(smp_rndm)))

print("Rand error = ", LA.norm(rhs(test) - np.dot(M, c_rndm), np.inf) / LA.norm(rhs(test), np.inf), "\n")
"""

In [ ]:
# visualization part
# position of the chosen points by MaxVol and GD in 2-D case
plt.xlim(l_bound-0.15, u_bound+0.15)
plt.ylim(l_bound-0.15, u_bound+0.15)
#plt.plot(M[row_indx,1], M[row_indx,2], 'ro', label = "MV")
plt.plot(pnts[:,0], pnts[:,1], 'ro', label = "GD")
#plt.plot(smp_sbl[:,0],smp_sbl[:,1], 'm*', label = "Sobol")
#plt.plot(smp[:,0],smp[:,1], 'go', label = "LHS")
#plt.plot(smp_rndm[:,0],smp_rndm[:,1], 'bx', label = "Random")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=4, borderaxespad=0.1)
plt.grid(True)
plt.show()

In [ ]:
log_det.NumOfClusters(pnts)

In [ ]:
# visualization of approximated function 
fig = plt.figure()
ax = fig.gca(projection='3d')

# make data
X = np.arange(l_bound, u_bound, 0.08)
Y = np.arange(l_bound, u_bound, 0.08)
X, Y = np.meshgrid(X, Y)

Z = 2*np.exp(-((X**2)/2. + (Y**2)/2.)) 
#Z = np.sin((X**2)/2. - (Y**2)/4. + 3) * np.cos(2*X + 1 - np.exp(Y))

# plot the surface
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0.03, antialiased=False)

# customize the z axis
# ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# add a color bar which maps values to colors
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

# Speed of d/dx

In [ ]:
num = 1000
xx = l_bound + (u_bound - l_bound)*np.random.rand(num, num_points*dim)
# g = grad(log_det.loss_func)
grad_f = grad(log_det.loss_func)
# grad_f = jit(g)

In [ ]:
%%time
for e in xx:
    grad_f(e)

In [ ]:
%%time
for e in xx:
    log_det.grad(e, False)